This file contains some functions to check that ICD OPERTN, MAINSPEF and PROCODES codes match with the reference tables. Functions are tested on APC tables (positive and negative). I assume the tables from different years are merged.

# Connection

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import numpy as np

%matplotlib inline

In [2]:
def connect():
    return pyodbc.connect(
        'DRIVER={/usr/lib/x86_64-linux-gnu/odbc/libtdsodbc.so}; SERVER=192.168.5.78;'
        'DATABASE=IMS-NHSDigital;UID=sa;PWD=2HsxpmkDLSlHlT;port=1433;'
        'TDS_Version=8.0;')
engine = sqlalchemy.create_engine('mssql+pyodbc://', creator=connect)
conn = engine.connect()

/home/vsalvatelli/anaconda3/lib/python3.6/site-packages/sqlalchemy/connectors/pyodbc.py:92: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  "No driver name specified; "


# Check on PROCODES

Extract the unique set of hospitals (PROCODE) codes and provide patient count for each. Join this with the reference table for the PROCODE codes and flag those that do not match a code in the reference table.

In [51]:
#extract codes
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

label_ID="PROCODE"

unique_procodes=set()
for tablename in table_list:
    query= 'SELECT ' + label_ID  +' from ' + tablename
    print(query)
    procodes=pd.read_sql(query, conn)[label_ID].unique()
    unique_procodes.update(procodes)
    print(len(unique_procodes))

#print(unique_procodes)
#I suggest to count for those not matching if we just care about them

SELECT PROCODE from dbo.NIC58999_APC_Linkage_201299
190
SELECT PROCODE from dbo.NIC58999_APC_Linkage_201399
244
SELECT PROCODE from dbo.NIC58999_APC_Linkage_201499
275
SELECT PROCODE from dbo.NIC58999_APC_Linkage_201599
296
SELECT PROCODE from dbo.NIC58999_APC_Linkage_201613
310


In [52]:
#reading reference codes
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimProvider.csv'
ref_label_procode='ProviderCode'
ref_procode=pd.read_csv(ref_table, delimiter="|")[ref_label_procode].unique()

/home/vsalvatelli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
len(set(ref_procode))

452

In [53]:
#some codes are not matching
unique_procodes.issubset(set(ref_procode))

False

In [61]:
#many codes not matching: am I reading the right col??
NM_procodes=[x for x in unique_procodes if x not in ref_procode]
print(len(NM_procodes)/len(unique_procodes))

0.8612903225806452
